In [2]:
setwd('/home/matt/MSOR/ISYE7406/ML-Ex-Rates/Results')
library(class)
library(MASS)
library(pls)
library(lars)
library(lattice)
library(dplyr)

In [3]:
#setwd('/home/matt/MSOR/ISYE7406/ML-Ex-Rates/Data')
#ex_rates <- read.csv('oecd.csv')
#ex_rates <- subset(ex_rates, ex_rates$Country.y == 'canada' |
#                  ex_rates$Country.y == 'europe' |
#                  ex_rates$Country.y == 'mexico' |
#                  ex_rates$Country.y == 'japan' |
#                  ex_rates$Country.y == 'korea')
#setwd('/home/matt/MSOR/ISYE7406/ML-Ex-Rates/Results')
#write.csv(ex_rates, file = 'topfive.csv')

In [4]:
country <- 'canada'
full <- read.csv(paste(country,'_full.csv',sep=''))
full <- full[,-1]
rownames(full) <- 1:nrow(full)
regmodel <- lm(Pct_Chg ~ ., data = full)
min.model = lm(Pct_Chg ~ 1, data = full)
max_model <- formula(regmodel)
linreg_AIC = step(min.model, direction = 'forward',
            scope = max_model, trace = FALSE)
vars <- as.character(formula(linreg_AIC))
var_list <- unlist(strsplit(vars[3],' +'))
var_list <- var_list[c(TRUE,FALSE)]
var_list <- c('Pct_Chg',var_list)
aic_vars <- full[var_list]
var_list

ERROR: Error in `[.data.frame`(full, var_list): undefined columns selected


[1] "Pct_Chg"           "Exchange"          "lag10.Exchange"   
 [4] "lag5.X1Y_Yield_f"  "FER_f"             "lag10.Infl_d"     
 [7] "lag2.Infl_f"       "lag2.Exchange"     "lag10.X1Y_Yield_f"
[10] "lag10.Int_f"       "Int_d"             "lag2.Int_f"       
[13] "lag6.Exchange"     "lag6.Infl_d"       "lag4.Int_d"       
[16] "lag10.Int_d"       "lag3.Int_f"        "lag2.X1Y_Yield_d" 
[19] "Int_f"             "lag3.Exchange"     "lag1.Exchange"    
[22] "GDPG_f"            "lag4.Infl_f"       "lag6.Infl_f"      
[25] "lag9.Infl_f"       "lag6.Int_d"        "lag5.X1Y_Yield_d" 
[28] "lag4.Int_f"        "lag7.Infl_f"       "lag8.Exchange"    
[31] "lag5.Infl_d"       "lag10.X1Y_Yield_d" "BOT_f"            
[34] "lag5.Infl_f"       "lag2.Int_d"        "lag9.Int_d"       
[37] "lag6.X1Y_Yield_f"  "\n"                "+"                
[40] "+"                 "+"                 "+"                
[43] "+"                 "+"                 "+"                
[46] "+"                 "+"                 "+"                
[49] "+"

In [3]:
test_lin_models <- function(country, B){
         
        full <- read.csv(paste(country,'_full.csv',sep=''))
        full <- full[,-1]
        rownames(full) <- 1:nrow(full)
    
        #Structural Model
        # struct <- subset(full, select = -c(GDPG_d,GDPG_f,BOT_f,BOT_d,FER_f))
    
        #AIC Model
        regmodel <- lm(Pct_Chg ~ ., data = full)
        min.model = lm(Pct_Chg ~ 1, data = full)
        max_model <- formula(regmodel)
        linreg_AIC = step(min.model, direction = 'forward',
                 scope = max_model, trace = FALSE)
        vars <- as.character(formula(linreg_AIC))
        var_list <- unlist(strsplit(vars[3],' +'))
        var_list <- var_list[c(TRUE,FALSE)]
        var_list <- c('Pct_Chg',var_list)
        aic_vars <- full[var_list]

        # Split into training and testing sets
        n <- dim(full)[1]
        n1 <- round(n/5)
        B <- 100
    
        # Test Full Model

        # Linear Regression -- Full Model
        set.seed(19890211)
        linreg.TE_list <- vector(length=0)
        for(i in 1:B){
            start <- sample(1:(n-n1-1),1)
            flag <- (start+6):(start+n1-6)            
            train <- aic_vars[-flag,]; test <- aic_vars[flag,]
    
            regmodel <- lm(Pct_Chg ~ ., data = train)
            linreg.test <- predict.lm(regmodel,test[,-1])
            linreg.TE <- mean( (test[,1]-linreg.test)^2)
            linreg.TE_list <- c(linreg.TE_list, linreg.TE)
    
        }    
    
        # Ridge Regression -- Full Model
        set.seed(19890211)
        ridge.TE_list <- vector(length=0)
        for(i in 1:B){
            start <- sample(1:(n-n1-1),1)
            flag <- (start+6):(start+n1-6)            
            train <- aic_vars[-flag,]; test <- aic_vars[flag,]
    
            ridgereg <- lm.ridge(Pct_Chg ~ ., data = train, 
                        lambda = seq(0,10,0.001))
            lambdaopt <- which.min(ridgereg$GCV)
            ridge_coef <- coef(ridgereg)[lambdaopt,]
            ridge_testmat <- as.matrix(cbind(1,test[,-1]))
            ridge.test <- as.matrix(ridge_testmat %*% ridge_coef)
            ridge.TE <- mean( (test[,1] - ridge.test)^2)
            ridge.TE_list <- c(ridge.TE_list, ridge.TE)
    
        }
    
        # Lasso Regression -- Full Model
        set.seed(19890211)
        lasso.TE_list <- vector(length=0)
        lasso_lambda <- 0
        for(i in 1:B){
            start <- sample(1:(n-n1-1),1)
            flag <- (start+6):(start+n1-6)    
            train <- aic_vars[-flag,]; test <- aic_vars[flag,]
    
            lassoreg <- lars( x= as.matrix(train[,-1]),
                            y = train[,1], type = 'lasso')
            Cp1 <- summary(lassoreg)$Cp
            index1 <- which.min(Cp1)
            lasso_coef <- coef(lassoreg)[index1,]
            if (is.na(lassoreg$lambda[index1])==FALSE){
                lasso_lambda <- lassoreg$lambda[index1]
            }
            lassoreg.fit <- predict(lassoreg, as.matrix(test[,-1]),
                                   s = lasso_lambda, type = 'fit',
                                   mode = 'lambda')
            lassoreg.test <- lassoreg.fit$fit
            lassoreg.TE <- mean( (test[,1]-lassoreg.test)^2)
            lasso.TE_list <- c(lasso.TE_list, lassoreg.TE)
                
        }
    
        # PCR -- Full Model
        set.seed(19890211)
        pcreg.TE_list <- vector(length=0)
        for(i in 1:B){
            start <- sample(1:(n-n1-1),1)
            flag <- (start+6):(start+n1-6)
            train <- aic_vars[-flag,]; test <- aic_vars[flag,]
            
            pcreg <- pcr(Pct_Chg ~ ., data = train,
                        validation = 'CV')
            comps <- dim(train)[2]-1
            pcreg.test <- predict(pcreg, ncomp = comps,
                                 newdata = test[,-1])
            pcreg.TE <- mean((test[,1]-pcreg.test)^2)
            pcreg.TE_list <- c(pcreg.TE_list, pcreg.TE)
        }
    
        # Naive
        set.seed(19890211)
        naive.TE_list <- vector(length=0)
        for(i in 1:B){
            start <- sample(1:(n-n1-1),1)
            flag <- (start+6):(start+n1-6)
            train <- full[-flag,]; test <- full[flag,]
            naive.TE <- mean ( (test[,1])^2)
            naive.TE_list <- c(naive.TE_list,naive.TE)
        }
    
        output1 <- c('Full','Linear','Ridge',
                    'Lasso', 'PCR','Naive')
        output1 <- rbind(output1, c('Mean', mean(linreg.TE_list),
                             mean(ridge.TE_list),mean(lasso.TE_list),
                             mean(pcreg.TE_list), mean(naive.TE_list)))
        output1 <- rbind(output1, c('Variance', var(linreg.TE_list),
                             var(ridge.TE_list), var(lasso.TE_list),
                             var(pcreg.TE_list),var(naive.TE_list)))
        output1 <- rbind(output1, c('t-test',
                t.test(linreg.TE_list,naive.TE_list,alternative='less',paired=TRUE)$p.value,
                t.test(ridge.TE_list,naive.TE_list,alternative='less',paired=TRUE)$p.value,
                t.test(lasso.TE_list,naive.TE_list,alternative='less',paired=TRUE)$p.value,
                t.test(pcreg.TE_list,naive.TE_list,alternative='less',paired=TRUE)$p.value, NA))
        output1 <- rbind(output1,c('t-test2',
                    NA,
                    t.test(ridge.TE_list,linreg.TE_list,paired=TRUE)$p.value,
                    t.test(lasso.TE_list,linreg.TE_list,paired=TRUE)$p.value,
                    t.test(pcreg.TE_list,linreg.TE_list,paired=TRUE)$p.value, NA))

        return(output1)
}

In [6]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}
print('canada')
test_lin_models('canada',100)
print('europe')
test_lin_models('europe',100)
print('mexico')
test_lin_models('mexico',100)
print('japan')
test_lin_models('japan',100)
print('korea')
test_lin_models('korea',100)

[1] "canada"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00186906625586332,0.00184553139431784,0.00184645004317239,0.00215179415951976,0.00633112124174333
,Variance,2.40808574481807e-06,2.71274614675143e-06,2.69066523933437e-06,3.21350254892735e-06,4.43202053185833e-05
,t-test,5.52268849964904e-13,3.40211514686072e-13,2.91993995329485e-13,9.31655464315176e-09,NA
,t-test2,NA,0.298719079987725,0.153976833472439,0.211562772267338,NA


[1] "europe"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0156745813042794,0.0161013200185915,0.0165834347378512,0.0215925632956085,0.0112421526834764
,Variance,0.000327848319593415,0.000347787446633328,0.000375838659336726,0.000600235656313522,3.98339802362644e-05
,t-test,0.992254468650509,0.995051952345602,0.99683383577635,0.999947539602393,NA
,t-test2,NA,4.15981373284092e-06,2.91644903438477e-06,0.0548733961007055,NA


[1] "mexico"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00312714146682264,0.00342753262728508,0.00332420712786075,0.00351063616811043,0.0108772389191061
,Variance,6.94421162744171e-06,1.1786671207418e-05,8.50186803511912e-06,8.2502450700009e-06,0.000171157819509417
,t-test,5.74615005164809e-10,2.590281974792e-10,9.28752585041708e-10,1.5957369480007e-07,NA
,t-test2,NA,0.00296893122673738,0.00141044415285232,0.339680309960024,NA


[1] "japan"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0437776480094219,0.0435090432502538,0.0392558635910223,0.0530695075949546,0.0202047523740304
,Variance,0.00109154313069146,0.00105842969301237,0.000725779932700122,0.00240848318388585,0.000113998123653752
,t-test,0.999999999962054,0.999999999965863,0.999999999966127,0.999999999180966,NA
,t-test2,NA,0.540574342160045,0.000243906621648604,0.0903705604356453,NA


[1] "korea"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00625696329637037,0.00607660900400986,0.00609882194386868,0.00620058711370977,0.0121161670932359
,Variance,2.00797921676752e-05,1.84094971584248e-05,1.78297285767447e-05,2.35181436757121e-05,0.000142648028891311
,t-test,1.90479749259429e-07,7.70648257732802e-08,5.75575149037917e-08,2.01240259115884e-05,NA
,t-test2,NA,1.75161188981925e-07,0.0160572318873937,0.93267945197668,NA


In [8]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}
print('canada')
test_lin_models('canada',100)
print('europe')
test_lin_models('europe',100)
print('mexico')
test_lin_models('mexico',100)
print('japan')
test_lin_models('japan',100)
print('korea')
test_lin_models('korea',100)

[1] "canada"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00358154677132368,0.00359890547323608,0.00362233997988435,0.00385239861302073,0.00648705127725761
,Variance,1.08761223846105e-05,1.12737076351513e-05,1.16421995295879e-05,1.35030758433384e-05,4.02867936526128e-05
,t-test,4.99537516248318e-11,3.63088602740568e-11,3.80697070484901e-11,0.000289345242207602,NA
,t-test2,NA,0.285706795217395,0.119994532693998,0.59649425676465,NA


[1] "europe"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0185970363263374,0.0185871728844995,0.0187216109818081,0.018906327387963,0.0111220819492909
,Variance,0.000679125844624423,0.000662138031644115,0.000661798902546887,0.000595404541751408,4.0795086542101e-05
,t-test,0.997899094260047,0.99813931199898,0.998422022278158,0.998638004733824,NA
,t-test2,NA,0.859659838912264,0.0810814735365217,0.917867224557536,NA


[1] "mexico"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00584158655216713,0.0058328753375973,0.00589145242155161,0.00478772468609226,0.0102881272943173
,Variance,4.31619959033225e-05,4.2113650897425e-05,4.51106746037937e-05,3.15644706954613e-05,0.000143533767634844
,t-test,3.55499393382868e-09,4.33139319420362e-09,2.00465926464418e-09,3.88091036914934e-05,NA
,t-test2,NA,0.580548809706549,0.733123430151565,0.225923901583925,NA


[1] "japan"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0459328173829366,0.0456377813092702,0.0455693302650388,0.0471524638003076,0.0201495909702087
,Variance,0.00116255991271125,0.00115473286073452,0.00114370075926039,0.00124258036440334,0.000111962472781096
,t-test,0.999999999998248,0.999999999997522,0.999999999997722,0.999999999994437,NA
,t-test2,NA,0.264208016906059,0.132318301931998,0.806833043263102,NA


[1] "korea"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.006838426058482,0.00671550320399052,0.00664821146413652,0.0062279726832557,0.0117719568713195
,Variance,2.89376434861646e-05,2.795240551806e-05,2.74617684993381e-05,1.69008565233865e-05,0.000135294840373022
,t-test,8.82696761980889e-07,4.29372824833916e-07,2.44978804245406e-07,1.71581130263148e-05,NA
,t-test2,NA,7.00554495466959e-07,0.000565884810735588,0.33855439116498,NA


In [9]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))


full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}
print('canada')
test_lin_models('canada',100)
print('europe')
test_lin_models('europe',100)
print('mexico')
test_lin_models('mexico',100)
print('japan')
test_lin_models('japan',100)
print('korea')
test_lin_models('korea',100)

[1] "canada"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00331800655151198,0.00332227501499223,0.00331800655151183,0.00434476291135303,0.00660709436015606
,Variance,9.84121920146836e-06,9.88678807940583e-06,9.84121920146866e-06,1.86797376322195e-05,3.98649989981387e-05
,t-test,1.50698402258061e-11,1.24938791189633e-11,1.50698402257652e-11,0.00256911006620651,NA
,t-test2,NA,0.700213129078403,0.0511922089566798,0.0714509796259312,NA


[1] "europe"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0147232143216596,0.0147860006055074,0.0155701526088764,0.0168837787949571,0.0110314766689762
,Variance,0.000248273215483387,0.000251224155008905,0.000284378248823494,0.000271523924901658,4.10034060554716e-05
,t-test,0.992917752188612,0.993594573156785,0.997714591482555,0.999231403592582,NA
,t-test2,NA,0.161189972769235,5.93546935262738e-06,0.366200679885619,NA


[1] "mexico"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00570276278938455,0.00571318966459037,0.00570847791687154,0.00455798742702769,0.0099987075276677
,Variance,5.16507605830181e-05,5.13042559342962e-05,5.2465224465651e-05,2.98540263877501e-05,0.000141899157035634
,t-test,1.13228897019804e-08,1.52524151690195e-08,1.30733779733266e-08,3.68929451916536e-05,NA
,t-test2,NA,0.274101062679827,0.807602804652563,0.203294095482441,NA


[1] "japan"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0474901194947727,0.0469970146192436,0.0467451650273824,0.0484692594747486,0.0200797780255698
,Variance,0.000786950018357243,0.000759082992930679,0.000743412265549126,0.00108875462665981,0.000109425524901833
,t-test,1,1,1,0.999999999999685,NA
,t-test2,NA,4.29266569263467e-07,0.00311838457518944,0.812487086543255,NA


[1] "korea"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00673945253826471,0.00665179656016668,0.00669107314928279,0.00535290626587546,0.0117620159072359
,Variance,2.82161823309336e-05,2.77456605667038e-05,2.79256224214658e-05,1.32709939089489e-05,0.000132006816519314
,t-test,4.13250195805013e-07,2.47498427537927e-07,2.88891041082084e-07,4.84025582311542e-07,NA
,t-test2,NA,1.69827842982397e-08,0.0079591070837313,0.0280696220515651,NA


In [5]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}
print('canada')
test_lin_models('canada',100)
print('europe')
test_lin_models('europe',100)
print('mexico')
test_lin_models('mexico',100)
print('japan')
test_lin_models('japan',100)
print('korea')
test_lin_models('korea',100)

countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}
print('canada')
test_lin_models('canada',100)
print('europe')
test_lin_models('europe',100)
print('mexico')
test_lin_models('mexico',100)
print('japan')
test_lin_models('japan',100)
print('korea')
test_lin_models('korea',100)

[1] "canada"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00137447142765265,0.00139509365331705,0.00140981593120451,0.00153293957752987,0.0063689539861169
,Variance,1.40773319104912e-06,1.60081267065167e-06,1.57810489541458e-06,2.19554109645504e-06,4.60193890284921e-05
,t-test,6.43982105020047e-13,4.90227720418683e-13,5.63326297024789e-13,1.81062209344964e-10,NA
,t-test2,NA,0.211702125748568,0.00821351736313249,0.374651498428165,NA


[1] "europe"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0159542418954869,0.0163539376076528,0.0166235239079376,0.0203443518510358,0.0112994204211395
,Variance,0.000336413356649337,0.000356487002993127,0.000335026109781091,0.000466901568691459,4.17729378045363e-05
,t-test,0.992186649661311,0.994736961628419,0.997358827107443,0.999933061970432,NA
,t-test2,NA,0.0357096824352124,0.050145554073907,0.121434255769427,NA


[1] "mexico"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0028615617515026,0.0027468464082027,0.0027108090345559,0.00238118991741538,0.0111549633168137
,Variance,5.58383385354546e-06,4.11880165097305e-06,3.60939619493282e-06,3.85064306965096e-06,0.000173602049772628
,t-test,2.60204702559401e-11,4.25575761544386e-11,7.04986757882319e-11,2.14375210951458e-09,NA
,t-test2,NA,0.0346862255737592,0.084759354866658,0.142804173272137,NA


[1] "japan"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0404320333588216,0.0398930189978332,0.0390227875229372,0.0494272851076801,0.0202686858054516
,Variance,0.00100895671754152,0.000996991530195429,0.000924464416449829,0.00207738347888312,0.000116316205648944
,t-test,0.999999996666163,0.99999999337539,0.999999991913354,0.999999992903298,NA
,t-test2,NA,0.133308363480293,0.00835226753038138,0.098297034666389,NA


[1] "korea"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00609018116065966,0.00597650219012279,0.0059096713299573,0.00486085871501267,0.012245580239268
,Variance,3.27018260866422e-05,3.14807870073193e-05,2.96427513043186e-05,1.54982810160556e-05,0.000148635062644459
,t-test,2.54979682696963e-11,1.69773359820543e-11,1.44004695379477e-11,2.50774434432188e-08,NA
,t-test2,NA,7.09576399125905e-10,0.000555387014365642,0.0620887185486345,NA


In [11]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4)) %>%
    
                mutate(lag5.Exchange = lag(Exchange,5)) %>%
                mutate(lag5.X1Y_Yield_d = lag(X1Y_Yield_d,5)) %>%
                mutate(lag5.X1Y_Yield_f = lag(X1Y_Yield_f,5)) %>%
                mutate(lag5.Infl_d = lag(Infl_d,5)) %>%
                mutate(lag5.Infl_f = lag(Infl_f,5)) %>%
                mutate(lag5.Int_d = lag(Int_d,5)) %>%
                mutate(lag5.Int_f = lag(Int_f,5))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}
print('canada')
test_lin_models('canada',100)
print('europe')
test_lin_models('europe',100)
print('mexico')
test_lin_models('mexico',100)
print('japan')
test_lin_models('japan',100)
print('korea')
test_lin_models('korea',100)

[1] "canada"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00160890932137478,0.0015284325425691,0.00158391142839407,0.0016874369367536,0.00648295339322649
,Variance,1.61776900573664e-06,1.72026428976833e-06,1.77131290264739e-06,2.06968775048527e-06,4.61856166968042e-05
,t-test,2.50949018827895e-12,1.86590692073055e-12,2.0710760120411e-12,5.53639597415778e-10,NA
,t-test2,NA,0.00216602411353764,0.0534417012570402,0.700907122230472,NA


[1] "europe"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0154799114613472,0.016063498445178,0.0162981767841719,0.0127137560535038,0.0113571665941031
,Variance,0.000274944551620313,0.000330353734316191,0.000336586068498566,0.000115088398679483,4.20839960517993e-05
,t-test,0.990197944833947,0.992926491379348,0.99475188530822,0.872410753576862,NA
,t-test2,NA,0.00481820651398803,0.00103056921035049,0.139571940567117,NA


[1] "mexico"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00141812914514133,0.00149971538114464,0.00173579819432141,0.00148396227985797,0.0114663924955909
,Variance,1.12618835105025e-06,1.03873530910738e-06,1.64187527032151e-06,9.47272028263573e-07,0.000190995884270143
,t-test,1.61555063302614e-11,1.83319945103005e-11,1.39914028082593e-11,7.63530047369959e-11,NA
,t-test2,NA,0.0001913470757057,3.60237519871321e-07,0.661208187835831,NA


[1] "japan"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0422426327007639,0.0412638757759306,0.0411594269102441,0.0451380727990496,0.0203196811623178
,Variance,0.00153052013377964,0.0014525123282971,0.00135977305311451,0.0016944675760946,0.000116319149682484
,t-test,0.99999992652905,0.999999870747119,0.999999939241073,0.99999994433545,NA
,t-test2,NA,0.0239244794362279,0.0104565220697581,0.614469724817589,NA


[1] "korea"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00515048787207996,0.0050568279979652,0.00507257405436347,0.00490153069034325,0.0123698454512141
,Variance,1.80789286534928e-05,1.77697669926927e-05,1.81355866411092e-05,1.53880003232725e-05,0.000149255975912202
,t-test,3.49495919726628e-11,1.9682171141991e-11,1.81187125841077e-11,4.09772462744802e-08,NA
,t-test2,NA,4.46540610968253e-09,0.00765692408770191,0.678971842025279,NA


In [12]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4)) %>%
    
                mutate(lag5.Exchange = lag(Exchange,5)) %>%
                mutate(lag5.X1Y_Yield_d = lag(X1Y_Yield_d,5)) %>%
                mutate(lag5.X1Y_Yield_f = lag(X1Y_Yield_f,5)) %>%
                mutate(lag5.Infl_d = lag(Infl_d,5)) %>%
                mutate(lag5.Infl_f = lag(Infl_f,5)) %>%
                mutate(lag5.Int_d = lag(Int_d,5)) %>%
                mutate(lag5.Int_f = lag(Int_f,5)) %>%
    
                mutate(lag6.Exchange = lag(Exchange,6)) %>%
                mutate(lag6.X1Y_Yield_d = lag(X1Y_Yield_d,6)) %>%
                mutate(lag6.X1Y_Yield_f = lag(X1Y_Yield_f,6)) %>%
                mutate(lag6.Infl_d = lag(Infl_d,6)) %>%
                mutate(lag6.Infl_f = lag(Infl_f,6)) %>%
                mutate(lag6.Int_d = lag(Int_d,6)) %>%
                mutate(lag6.Int_f = lag(Int_f,6))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}
print('canada')
test_lin_models('canada',100)
print('europe')
test_lin_models('europe',100)
print('mexico')
test_lin_models('mexico',100)
print('japan')
test_lin_models('japan',100)
print('korea')
test_lin_models('korea',100)

[1] "canada"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00144229494182569,0.00144398831579446,0.0014322628592925,0.0018288261178748,0.00670059744547656
,Variance,1.60555699158633e-06,1.72283184913806e-06,1.66988023234133e-06,2.39738191673901e-06,4.91969984023784e-05
,t-test,7.47179512227349e-13,6.20154095657846e-13,6.78722830292751e-13,4.61307073522667e-10,NA
,t-test2,NA,0.896620212054236,0.346806114203689,0.0463251876829853,NA


[1] "europe"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0125623762233728,0.0126943818584806,0.012943127617077,0.0110999501653876,0.0114253898494923
,Variance,0.000144572127714765,0.000143306158901065,0.000147453281536806,0.000131573771182163,4.23292101401763e-05
,t-test,0.799901266820532,0.827530997998031,0.868143672871657,0.407402420698107,NA
,t-test2,NA,0.207808388792406,0.00319642840541631,0.394442701017187,NA


[1] "mexico"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00126408793276698,0.00136981274070179,0.00136882228510743,0.00153303069852684,0.0117000401326953
,Variance,8.046014666536e-07,8.65397290924682e-07,8.11272995466213e-07,1.55727085430821e-06,0.000199443678546521
,t-test,1.16480889663632e-11,2.36490292207311e-11,2.70720172652146e-11,7.32576453154711e-11,NA
,t-test2,NA,0.00487806051203789,0.0586006073858747,0.0853456865871774,NA


[1] "japan"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0363874011541485,0.0357933347478722,0.0314817868837232,0.036856724404807,0.0203737779704506
,Variance,0.00153418219448639,0.00150977810295397,0.000635369627795392,0.00199952227250066,0.000118784630420302
,t-test,0.999961656609921,0.999938620281194,0.999982850735033,0.999782666006934,NA
,t-test2,NA,0.0583843502513162,0.00667934827162595,0.931651847944092,NA


[1] "korea"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00550160426682836,0.00534354251884261,0.00538065736918586,0.00480240080855968,0.0124884865199863
,Variance,2.18600908580342e-05,2.13860505615792e-05,2.16912594207169e-05,1.66280547544528e-05,0.000156272004295333
,t-test,1.99547355666215e-11,9.51483385563492e-12,9.89756911031309e-12,5.1543362518875e-08,NA
,t-test2,NA,7.91371793002353e-17,2.26939618773107e-06,0.275695962706508,NA


In [13]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4)) %>%
    
                mutate(lag5.Exchange = lag(Exchange,5)) %>%
                mutate(lag5.X1Y_Yield_d = lag(X1Y_Yield_d,5)) %>%
                mutate(lag5.X1Y_Yield_f = lag(X1Y_Yield_f,5)) %>%
                mutate(lag5.Infl_d = lag(Infl_d,5)) %>%
                mutate(lag5.Infl_f = lag(Infl_f,5)) %>%
                mutate(lag5.Int_d = lag(Int_d,5)) %>%
                mutate(lag5.Int_f = lag(Int_f,5)) %>%
    
                mutate(lag6.Exchange = lag(Exchange,6)) %>%
                mutate(lag6.X1Y_Yield_d = lag(X1Y_Yield_d,6)) %>%
                mutate(lag6.X1Y_Yield_f = lag(X1Y_Yield_f,6)) %>%
                mutate(lag6.Infl_d = lag(Infl_d,6)) %>%
                mutate(lag6.Infl_f = lag(Infl_f,6)) %>%
                mutate(lag6.Int_d = lag(Int_d,6)) %>%
                mutate(lag6.Int_f = lag(Int_f,6)) %>%

                mutate(lag7.Exchange = lag(Exchange,7)) %>%
                mutate(lag7.X1Y_Yield_d = lag(X1Y_Yield_d,7)) %>%
                mutate(lag7.X1Y_Yield_f = lag(X1Y_Yield_f,7)) %>%
                mutate(lag7.Infl_d = lag(Infl_d,7)) %>%
                mutate(lag7.Infl_f = lag(Infl_f,7)) %>%
                mutate(lag7.Int_d = lag(Int_d,7)) %>%
                mutate(lag7.Int_f = lag(Int_f,7))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}
print('canada')
test_lin_models('canada',100)
print('europe')
test_lin_models('europe',100)
print('mexico')
test_lin_models('mexico',100)
print('japan')
test_lin_models('japan',100)
print('korea')
test_lin_models('korea',100)

[1] "canada"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00123911389098851,0.00125259032042335,0.0013525487169712,0.00147023816642057,0.00681916504081449
,Variance,1.31366049731978e-06,1.44208875894938e-06,1.73654796171014e-06,1.72608293284479e-06,5.05065856740398e-05
,t-test,1.70088904085548e-13,1.36125183630198e-13,1.12993726934842e-13,7.6138533720592e-12,NA
,t-test2,NA,0.146172898043094,6.72915729123738e-05,0.207176299935249,NA


[1] "europe"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00764830089504619,0.00774268079111581,0.00771839070062704,0.00806832887499269,0.0114919568392703
,Variance,3.00653568199331e-05,3.15212943330229e-05,2.90228756134128e-05,3.88520297376259e-05,4.21606090625108e-05
,t-test,5.88386839752639e-06,7.22998677596608e-06,6.07541949046988e-06,5.83576564014392e-05,NA
,t-test2,NA,0.3906964978114,0.163977166712341,0.575851370758091,NA


[1] "mexico"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00151627162822287,0.0014217402301535,0.00160895373683857,0.0022548663987722,0.011751901722461
,Variance,2.21345615522915e-06,1.67802083585413e-06,2.0113030963563e-06,5.34590201626506e-06,0.000204362831271738
,t-test,1.2307122760777e-11,1.72465216718842e-11,3.68792028648038e-11,2.74341941136588e-09,NA
,t-test2,NA,0.0188689048444495,0.220024385408619,0.00915292935357984,NA


[1] "japan"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0236656928334346,0.0234575303885714,0.0224000599783584,0.0273165520227508,0.0204509646616379
,Variance,0.000328439236506042,0.000327899846776385,0.000305256516166296,0.00056946663771329,0.000118465580668487
,t-test,0.948052199684183,0.935565449778954,0.845516621865929,0.992678405090459,NA
,t-test2,NA,0.130359454919513,0.000108270883765645,0.21221430252164,NA


[1] "korea"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00512914275265451,0.00503746732987676,0.00511504085873419,0.00560056079326607,0.0126349545663202
,Variance,2.13621806256392e-05,2.1013117112158e-05,2.11710125661629e-05,2.52269825395731e-05,0.000165313442015468
,t-test,6.19520697713211e-12,4.405613645309e-12,6.09828526176918e-12,1.33160864397335e-06,NA
,t-test2,NA,1.78974725975541e-09,0.634136102480477,0.503301495124489,NA


In [14]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4)) %>%
    
                mutate(lag5.Exchange = lag(Exchange,5)) %>%
                mutate(lag5.X1Y_Yield_d = lag(X1Y_Yield_d,5)) %>%
                mutate(lag5.X1Y_Yield_f = lag(X1Y_Yield_f,5)) %>%
                mutate(lag5.Infl_d = lag(Infl_d,5)) %>%
                mutate(lag5.Infl_f = lag(Infl_f,5)) %>%
                mutate(lag5.Int_d = lag(Int_d,5)) %>%
                mutate(lag5.Int_f = lag(Int_f,5)) %>%
    
                mutate(lag6.Exchange = lag(Exchange,6)) %>%
                mutate(lag6.X1Y_Yield_d = lag(X1Y_Yield_d,6)) %>%
                mutate(lag6.X1Y_Yield_f = lag(X1Y_Yield_f,6)) %>%
                mutate(lag6.Infl_d = lag(Infl_d,6)) %>%
                mutate(lag6.Infl_f = lag(Infl_f,6)) %>%
                mutate(lag6.Int_d = lag(Int_d,6)) %>%
                mutate(lag6.Int_f = lag(Int_f,6)) %>%

                mutate(lag7.Exchange = lag(Exchange,7)) %>%
                mutate(lag7.X1Y_Yield_d = lag(X1Y_Yield_d,7)) %>%
                mutate(lag7.X1Y_Yield_f = lag(X1Y_Yield_f,7)) %>%
                mutate(lag7.Infl_d = lag(Infl_d,7)) %>%
                mutate(lag7.Infl_f = lag(Infl_f,7)) %>%
                mutate(lag7.Int_d = lag(Int_d,7)) %>%
                mutate(lag7.Int_f = lag(Int_f,7)) %>%

                mutate(lag8.Exchange = lag(Exchange,8)) %>%
                mutate(lag8.X1Y_Yield_d = lag(X1Y_Yield_d,8)) %>%
                mutate(lag8.X1Y_Yield_f = lag(X1Y_Yield_f,8)) %>%
                mutate(lag8.Infl_d = lag(Infl_d,8)) %>%
                mutate(lag8.Infl_f = lag(Infl_f,8)) %>%
                mutate(lag8.Int_d = lag(Int_d,8)) %>%
                mutate(lag8.Int_f = lag(Int_f,8))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}
print('canada')
test_lin_models('canada',100)
print('europe')
test_lin_models('europe',100)
print('mexico')
test_lin_models('mexico',100)
print('japan')
test_lin_models('japan',100)
print('korea')
test_lin_models('korea',100)

[1] "canada"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00100597597473417,0.00103261838057623,0.00101433915995499,0.00112978268354799,0.00711641477391088
,Variance,6.27308383808294e-07,7.33146784074877e-07,7.07621446667268e-07,1.01664534068017e-06,5.35428692990199e-05
,t-test,4.30297675645299e-14,3.04101430216719e-14,3.67512799159629e-14,8.1552360442554e-13,NA
,t-test2,NA,0.0602596009919794,0.381036143891707,0.313121747141786,NA


[1] "europe"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00661259983390063,0.0070161189899421,0.00715760058216049,0.00595731952004099,0.011530186050274
,Variance,2.43442005465933e-05,2.97248380212546e-05,2.96968050456168e-05,2.34214805340914e-05,4.28845745725253e-05
,t-test,9.78648586879478e-09,1.22754679766722e-07,2.57379756475911e-07,5.60657239774106e-11,NA
,t-test2,NA,2.45680050950783e-06,1.59103272570801e-09,0.381484632657068,NA


[1] "mexico"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00133361782938726,0.00135117870491115,0.0014894953114133,0.00151352204953439,0.0118078650402035
,Variance,2.06212059319412e-06,2.39239644046154e-06,2.11607903867474e-06,5.35455000892331e-06,0.000218113669761617
,t-test,2.83605312254138e-11,3.18476639031093e-11,4.91763262628297e-11,4.74114130294915e-10,NA
,t-test2,NA,0.635311622753784,0.00383872819689129,0.524066598329169,NA


[1] "japan"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0168655530568457,0.0166379597371702,0.0163414066429113,0.0211271607600003,0.0205393979190816
,Variance,0.000134878967618576,0.000134871239940392,0.000116339886171879,0.000583022584258184,0.000120974360925535
,t-test,0.00459297996117838,0.0030731691333932,0.00118031838066083,0.588494351143495,NA
,t-test2,NA,0.107359509829361,0.0488517426640045,0.126026291693994,NA


[1] "korea"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00451942049886738,0.00437164536364006,0.00455544428534455,0.00390057390396877,0.0128344062186203
,Variance,2.02850777440439e-05,1.86867368494769e-05,1.98014492569187e-05,1.11102482313953e-05,0.000166370349049541
,t-test,2.22572576366307e-14,1.17647868443249e-14,1.70680657195727e-14,2.9665423237785e-10,NA
,t-test2,NA,6.47157931459446e-05,0.432112471124535,0.240953800331006,NA


In [15]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4)) %>%
    
                mutate(lag5.Exchange = lag(Exchange,5)) %>%
                mutate(lag5.X1Y_Yield_d = lag(X1Y_Yield_d,5)) %>%
                mutate(lag5.X1Y_Yield_f = lag(X1Y_Yield_f,5)) %>%
                mutate(lag5.Infl_d = lag(Infl_d,5)) %>%
                mutate(lag5.Infl_f = lag(Infl_f,5)) %>%
                mutate(lag5.Int_d = lag(Int_d,5)) %>%
                mutate(lag5.Int_f = lag(Int_f,5)) %>%
    
                mutate(lag6.Exchange = lag(Exchange,6)) %>%
                mutate(lag6.X1Y_Yield_d = lag(X1Y_Yield_d,6)) %>%
                mutate(lag6.X1Y_Yield_f = lag(X1Y_Yield_f,6)) %>%
                mutate(lag6.Infl_d = lag(Infl_d,6)) %>%
                mutate(lag6.Infl_f = lag(Infl_f,6)) %>%
                mutate(lag6.Int_d = lag(Int_d,6)) %>%
                mutate(lag6.Int_f = lag(Int_f,6)) %>%

                mutate(lag7.Exchange = lag(Exchange,7)) %>%
                mutate(lag7.X1Y_Yield_d = lag(X1Y_Yield_d,7)) %>%
                mutate(lag7.X1Y_Yield_f = lag(X1Y_Yield_f,7)) %>%
                mutate(lag7.Infl_d = lag(Infl_d,7)) %>%
                mutate(lag7.Infl_f = lag(Infl_f,7)) %>%
                mutate(lag7.Int_d = lag(Int_d,7)) %>%
                mutate(lag7.Int_f = lag(Int_f,7)) %>%

                mutate(lag8.Exchange = lag(Exchange,8)) %>%
                mutate(lag8.X1Y_Yield_d = lag(X1Y_Yield_d,8)) %>%
                mutate(lag8.X1Y_Yield_f = lag(X1Y_Yield_f,8)) %>%
                mutate(lag8.Infl_d = lag(Infl_d,8)) %>%
                mutate(lag8.Infl_f = lag(Infl_f,8)) %>%
                mutate(lag8.Int_d = lag(Int_d,8)) %>%
                mutate(lag8.Int_f = lag(Int_f,8)) %>%
    
                mutate(lag9.Exchange = lag(Exchange,9)) %>%
                mutate(lag9.X1Y_Yield_d = lag(X1Y_Yield_d,9)) %>%
                mutate(lag9.X1Y_Yield_f = lag(X1Y_Yield_f,9)) %>%
                mutate(lag9.Infl_d = lag(Infl_d,9)) %>%
                mutate(lag9.Infl_f = lag(Infl_f,9)) %>%
                mutate(lag9.Int_d = lag(Int_d,9)) %>%
                mutate(lag9.Int_f = lag(Int_f,9))

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}
print('canada')
test_lin_models('canada',100)
print('europe')
test_lin_models('europe',100)
print('mexico')
test_lin_models('mexico',100)
print('japan')
test_lin_models('japan',100)
print('korea')
test_lin_models('korea',100)

[1] "canada"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.000608183145705694,0.000656017620166459,0.000673574517444733,0.000860121837469591,0.00714056502238315
,Variance,2.02371591464068e-07,3.2106316860442e-07,3.18238395387662e-07,5.29854174944236e-07,5.31926670863628e-05
,t-test,4.59521204017755e-15,3.91206230695146e-15,4.09467360714014e-15,7.36208889675232e-14,NA
,t-test2,NA,0.00209573051288047,0.000461364538016455,0.00155612052919797,NA


[1] "europe"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00415081167615026,0.00438946328170402,0.00434360743543172,0.00386934101265669,0.0115927011512398
,Variance,6.08347272800817e-06,7.39114213111612e-06,6.86050015398912e-06,5.99663516846219e-06,4.57675639981727e-05
,t-test,2.46909486864001e-18,4.33108626281103e-18,3.97996805524047e-18,4.97806521517191e-19,NA
,t-test2,NA,1.76002091648824e-07,3.21732512256471e-07,0.440218701207076,NA


[1] "mexico"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00106413978289238,0.00121736730953445,0.00118693643667644,0.00129518887643392,0.0123758871163262
,Variance,1.62267779606128e-06,1.42139662989024e-06,1.85103746564007e-06,2.27491428107715e-06,0.000225698215288191
,t-test,3.32890434602837e-12,5.94881108501039e-12,5.01295629534888e-12,5.16923791327605e-11,NA
,t-test2,NA,0.0144240512240167,0.0183085697412533,0.273081788622433,NA


[1] "japan"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00984706495614857,0.00984018548984247,0.00962208403305904,0.00960246305993554,0.0205895087693659
,Variance,2.52410629816181e-05,2.45134900061627e-05,2.07012091129938e-05,2.54592879333713e-05,0.000124534464154342
,t-test,8.26534341207276e-15,7.28426398156494e-15,1.3005234716404e-15,2.36915645877713e-14,NA
,t-test2,NA,0.495043656652983,0.00133690095438805,0.725231231021155,NA


[1] "korea"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00291995903106997,0.0029316964969929,0.00310904293670996,0.00251949906451078,0.0128872430697519
,Variance,4.19785408942282e-06,4.71137913415253e-06,5.60878953269898e-06,3.30748573111884e-06,0.000175975364851381
,t-test,1.07622059249693e-13,7.53665755083542e-14,7.70415837472509e-14,2.54121097713649e-12,NA
,t-test2,NA,0.547798436305724,7.78682044352837e-06,0.150291996615071,NA


In [16]:
countries <- c('japan','europe','korea','mexico','canada')
for (country in countries){
ex_rates <- read.csv('topfive.csv')
ex_rates <- subset(ex_rates, ex_rates$Country.y==country)
    
full <- subset(ex_rates, select = c(Pct_Chg, Exchange, X1Y_Yield_d, 
                X1Y_Yield_f, Int_d, Int_f,  Infl_d, Infl_f, GDPG_d,
                GDPG_f, BOT_f, BOT_d, FER_f))

full <- full %>% mutate(lag1.Exchange = lag(Exchange,1)) %>%
                mutate(lag1.X1Y_Yield_d = lag(X1Y_Yield_d,1)) %>%
                mutate(lag1.X1Y_Yield_f = lag(X1Y_Yield_f,1)) %>%
                mutate(lag1.Infl_d = lag(Infl_d,1)) %>%
                mutate(lag1.Infl_f = lag(Infl_f,1)) %>%
                mutate(lag1.Int_d = lag(Int_d,1)) %>%
                mutate(lag1.Int_f = lag(Int_f,1)) %>%

                mutate(lag2.Exchange = lag(Exchange,2)) %>%
                mutate(lag2.X1Y_Yield_d = lag(X1Y_Yield_d,2)) %>%
                mutate(lag2.X1Y_Yield_f = lag(X1Y_Yield_f,2)) %>%
                mutate(lag2.Infl_d = lag(Infl_d,2)) %>%
                mutate(lag2.Infl_f = lag(Infl_f,2)) %>%
                mutate(lag2.Int_d = lag(Int_d,2)) %>%
                mutate(lag2.Int_f = lag(Int_f,2)) %>%
    
                mutate(lag3.Exchange = lag(Exchange,3)) %>%
                mutate(lag3.X1Y_Yield_d = lag(X1Y_Yield_d,3)) %>%
                mutate(lag3.X1Y_Yield_f = lag(X1Y_Yield_f,3)) %>%
                mutate(lag3.Infl_d = lag(Infl_d,3)) %>%
                mutate(lag3.Infl_f = lag(Infl_f,3)) %>%
                mutate(lag3.Int_d = lag(Int_d,3)) %>%
                mutate(lag3.Int_f = lag(Int_f,3)) %>%
    
                mutate(lag4.Exchange = lag(Exchange,4)) %>%
                mutate(lag4.X1Y_Yield_d = lag(X1Y_Yield_d,4)) %>%
                mutate(lag4.X1Y_Yield_f = lag(X1Y_Yield_f,4)) %>%
                mutate(lag4.Infl_d = lag(Infl_d,4)) %>%
                mutate(lag4.Infl_f = lag(Infl_f,4)) %>%
                mutate(lag4.Int_d = lag(Int_d,4)) %>%
                mutate(lag4.Int_f = lag(Int_f,4)) %>%
    
                mutate(lag5.Exchange = lag(Exchange,5)) %>%
                mutate(lag5.X1Y_Yield_d = lag(X1Y_Yield_d,5)) %>%
                mutate(lag5.X1Y_Yield_f = lag(X1Y_Yield_f,5)) %>%
                mutate(lag5.Infl_d = lag(Infl_d,5)) %>%
                mutate(lag5.Infl_f = lag(Infl_f,5)) %>%
                mutate(lag5.Int_d = lag(Int_d,5)) %>%
                mutate(lag5.Int_f = lag(Int_f,5)) %>%
    
                mutate(lag6.Exchange = lag(Exchange,6)) %>%
                mutate(lag6.X1Y_Yield_d = lag(X1Y_Yield_d,6)) %>%
                mutate(lag6.X1Y_Yield_f = lag(X1Y_Yield_f,6)) %>%
                mutate(lag6.Infl_d = lag(Infl_d,6)) %>%
                mutate(lag6.Infl_f = lag(Infl_f,6)) %>%
                mutate(lag6.Int_d = lag(Int_d,6)) %>%
                mutate(lag6.Int_f = lag(Int_f,6)) %>%

                mutate(lag7.Exchange = lag(Exchange,7)) %>%
                mutate(lag7.X1Y_Yield_d = lag(X1Y_Yield_d,7)) %>%
                mutate(lag7.X1Y_Yield_f = lag(X1Y_Yield_f,7)) %>%
                mutate(lag7.Infl_d = lag(Infl_d,7)) %>%
                mutate(lag7.Infl_f = lag(Infl_f,7)) %>%
                mutate(lag7.Int_d = lag(Int_d,7)) %>%
                mutate(lag7.Int_f = lag(Int_f,7)) %>%

                mutate(lag8.Exchange = lag(Exchange,8)) %>%
                mutate(lag8.X1Y_Yield_d = lag(X1Y_Yield_d,8)) %>%
                mutate(lag8.X1Y_Yield_f = lag(X1Y_Yield_f,8)) %>%
                mutate(lag8.Infl_d = lag(Infl_d,8)) %>%
                mutate(lag8.Infl_f = lag(Infl_f,8)) %>%
                mutate(lag8.Int_d = lag(Int_d,8)) %>%
                mutate(lag8.Int_f = lag(Int_f,8)) %>%
    
                mutate(lag9.Exchange = lag(Exchange,9)) %>%
                mutate(lag9.X1Y_Yield_d = lag(X1Y_Yield_d,9)) %>%
                mutate(lag9.X1Y_Yield_f = lag(X1Y_Yield_f,9)) %>%
                mutate(lag9.Infl_d = lag(Infl_d,9)) %>%
                mutate(lag9.Infl_f = lag(Infl_f,9)) %>%
                mutate(lag9.Int_d = lag(Int_d,9)) %>%
                mutate(lag9.Int_f = lag(Int_f,9)) %>%
    
                mutate(lag10.Exchange = lag(Exchange,10)) %>%
                mutate(lag10.X1Y_Yield_d = lag(X1Y_Yield_d,10)) %>%
                mutate(lag10.X1Y_Yield_f = lag(X1Y_Yield_f,10)) %>%
                mutate(lag10.Infl_d = lag(Infl_d,10)) %>%
                mutate(lag10.Infl_f = lag(Infl_f,10)) %>%
                mutate(lag10.Int_d = lag(Int_d,10)) %>%
                mutate(lag10.Int_f = lag(Int_f,10)) 

full <- na.omit(full)
write.csv(full,file = paste(country,'_full.csv',sep=''))
}
print('canada')
test_lin_models('canada',100)
print('europe')
test_lin_models('europe',100)
print('mexico')
test_lin_models('mexico',100)
print('japan')
test_lin_models('japan',100)
print('korea')
test_lin_models('korea',100)

[1] "canada"


ERROR: Error in `[.data.frame`(full, var_list): undefined columns selected


[1] "europe"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.00250316012843876,0.00264955904746569,0.00261753341347395,0.00249030754860354,0.0116114696976192
,Variance,1.6304746195574e-06,2.1423356050675e-06,1.73107140969807e-06,1.66162478048471e-06,4.61538070756363e-05
,t-test,5.29029681791968e-25,1.40165122176932e-24,5.98814289086285e-25,9.18570072707343e-25,NA
,t-test2,NA,1.2695241375623e-08,0.000315315794111931,0.945043088800977,NA


[1] "mexico"


ERROR: Error in `[.data.frame`(full, var_list): undefined columns selected


[1] "japan"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0086208388816652,0.00850848412606427,0.00813950966843622,0.00996685634185489,0.0207024292873681
,Variance,3.32190977133562e-05,3.1088926405582e-05,2.61295046158068e-05,7.08120091110626e-05,0.000123930121839477
,t-test,3.61468751761268e-17,1.87335028826422e-17,3.26484690088587e-18,7.52519546370764e-12,NA
,t-test2,NA,0.0226980874806721,7.11657930426578e-05,0.176417773319081,NA


[1] "korea"


output1,Full,Linear,Ridge,Lasso,PCR,Naive
,Mean,0.0019103270937254,0.00193187143279267,0.00196764107371491,0.00190861606630818,0.013032439513106
,Variance,1.66418395737384e-06,1.97953656963217e-06,1.98703348395765e-06,2.17148148836712e-06,0.000175213470888662
,t-test,6.728501645096e-15,4.75646217429114e-15,5.42743901407179e-15,2.19650807779957e-13,NA
,t-test2,NA,0.160334309820133,0.0025437649306594,0.993132354517229,NA
